In [ ]:
## test 
## 1. anchor_boxes
## 2. groudtruth encode
## 3. bboxes decode(not yet finished)

In [10]:
import numpy as np
import math
import tensorflow as tf
import sys
sys.path.insert(0,'../processing/')
sys.path.insert(0,'../')
from image_processing2 import *
import tf_extended as tfe

In [3]:
img_shape=(300, 300)
num_classes=2
feat_layers=['conv_4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'pool6']
feat_shapes=[(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]
scale_range=[0.20, 0.90]
anchor_ratios=[1,2,3,5,7,10]
normalizations=[20, -1, -1, -1, -1, -1]
prior_scaling=[0.1, 0.1, 0.2, 0.2]

step = (scale_range[1] - scale_range[0]) / len(feat_shapes)
scales = [scale_range[0] + i * step for i in range(len(feat_shapes)+1)]
print scales

[0.2, 0.31666666666666665, 0.43333333333333335, 0.55, 0.6666666666666666, 0.7833333333333332, 0.8999999999999999]


In [4]:
def textbox_anchor_one_layer(img_shape,
                             feat_size,
                             ratios,
                             scale,
                             offset = 0.5,
                             dtype=np.float32):
    # Follow the papers scheme
    # 12 ahchor boxes with out sk' = sqrt(sk * sk+1)
    y, x = np.mgrid[0:feat_size[0], 0:feat_size[1]] + 0.5
    y = y.astype(dtype) / feat_size[0]
    x = x.astype(dtype) / feat_size[1]

    x_offset = x
    y_offset = y + offset
    x_out = np.stack((x, x_offset), -1)
    y_out = np.stack((y, y_offset), -1)
    y_out = np.expand_dims(y_out, axis=-1)
    x_out = np.expand_dims(x_out, axis=-1)

    # 
    num_anchors = 6
    h = np.zeros((num_anchors, ), dtype=dtype)
    w = np.zeros((num_anchors, ), dtype=dtype)
    for i ,r in enumerate(ratios):
        h[i] = scale / math.sqrt(r) / feat_size[0]
        w[i] = scale * math.sqrt(r) / feat_size[1]
    return y_out, x_out, h, w

y,x,h,w = textbox_anchor_one_layer((300,300), (38,38),(1,2,3,5,7,10),0.9)
print y.shape
print h
print (y -h).shape

(38, 38, 2, 1)
[ 0.02368421  0.01674727  0.01367409  0.0105919   0.00895179  0.0074896 ]
(38, 38, 2, 6)


In [5]:
def textbox_achor_all_layers(img_shape,
                           layers_shape,
                           anchor_ratios,
                           scales,
                           offset=0.5,
                           dtype=np.float32):
    """
    Compute anchor boxes for all feature layers.
    """
    layers_anchors = []
    for i, s in enumerate(layers_shape):
        anchor_bboxes = textbox_anchor_one_layer(img_shape, s,
                                                 anchor_ratios,
                                                 scales[i],
                                                 offset=offset, dtype=dtype)
        layers_anchors.append(anchor_bboxes)
    return layers_anchors

layers_anchors = textbox_achor_all_layers((300,300), feat_shapes,anchor_ratios,scales)
print len(layers_anchors)
print len(layers_anchors[0])
print layers_anchors[0][0].shape


6
4
(38, 38, 2, 1)


In [12]:
# =========================================================================== #
# TensorFlow implementation of Text Boxes encoding / decoding.
# =========================================================================== #

def tf_text_bboxes_encode_layer(bboxes,
                               anchors_layer,
                               matching_threshold=0.5,
                               prior_scaling=[0.1, 0.1, 0.2, 0.2],
                               dtype=tf.float32):
    
    """
    Encode groundtruth labels and bounding boxes using Textbox anchors from
    one layer.

    Arguments:
      bboxes: Nx4 Tensor(float) with bboxes relative coordinates;
      anchors_layer: Numpy array with layer anchors;
      matching_threshold: Threshold for positive match with groundtruth bboxes;
      prior_scaling: Scaling of encoded coordinates.

    Return:
      (target_localizations, target_scores): Target Tensors.
    # thisi is a binary problem, so target_score and tartget_labels are same.
    """
    # Anchors coordinates and volume.

    yref, xref, href, wref = anchors_layer
    print yref.shape
    print href.shape
    print bboxes.shape
    ymin = yref - href / 2.
    xmin = xref - wref / 2.
    ymax = yref + href / 2.
    xmax = xref + wref / 2. 
    vol_anchors = (xmax - xmin) * (ymax - ymin)
    
    # Initialize tensors...
    shape = (yref.shape[0], yref.shape[1], yref.shape[2], href.size)
    # all follow the shape(feat.size, feat.size, 2, 6)
    #feat_labels = tf.zeros(shape, dtype=tf.int64)
    feat_scores = tf.zeros(shape, dtype=dtype)

    feat_ymin = tf.zeros(shape, dtype=dtype)
    feat_xmin = tf.zeros(shape, dtype=dtype)
    feat_ymax = tf.ones(shape, dtype=dtype)
    feat_xmax = tf.ones(shape, dtype=dtype)

    def jaccard_with_anchors(bbox):
        """
        Compute jaccard score between a box and the anchors.
        """
        int_ymin = tf.maximum(ymin, bbox[0])
        int_xmin = tf.maximum(xmin, bbox[1])
        int_ymax = tf.minimum(ymax, bbox[2])
        int_xmax = tf.minimum(xmax, bbox[3])
        h = tf.maximum(int_ymax - int_ymin, 0.)
        w = tf.maximum(int_xmax - int_xmin, 0.)
        # Volumes.
        inter_vol = h * w
        union_vol = vol_anchors - inter_vol \
            + (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        jaccard = tf.div(inter_vol, union_vol)
        return jaccard
    
    """
    # never use in Textbox
    def intersection_with_anchors(bbox):
        '''
        Compute intersection between score a box and the anchors.
        '''
        int_ymin = tf.maximum(ymin, bbox[0])
        int_xmin = tf.maximum(xmin, bbox[1])
        int_ymax = tf.minimum(ymax, bbox[2])
        int_xmax = tf.minimum(xmax, bbox[3])
        h = tf.maximum(int_ymax - int_ymin, 0.)
        w = tf.maximum(int_xmax - int_xmin, 0.)
        inter_vol = h * w
        scores = tf.div(inter_vol, vol_anchors)
        return scores
    """
    
    def condition(i, feat_scores,
                  feat_ymin, feat_xmin, feat_ymax, feat_xmax):
        """Condition: check label index.
        """
        r = tf.less(i, 3)
        return r

    def body(i, feat_scores,feat_ymin, feat_xmin, feat_ymax, feat_xmax,bbox):
        """Body: update feature labels, scores and bboxes.
        Follow the original SSD paper for that purpose:
          - assign values when jaccard > 0.5;
          - only update if beat the score of other bboxes.
        """
        # Jaccard score.
        #bbox = bboxes[i]
        jaccard = jaccard_with_anchors(bbox)
        # Mask: check threshold + scores + no annotations + num_classes.
        mask = tf.greater(jaccard, feat_scores)
        mask = tf.logical_and(mask, tf.greater(jaccard, matching_threshold))
        #mask = tf.logical_and(mask, feat_scores > -0.5)
        #mask = tf.logical_and(mask, label < num_classes)
        imask = tf.cast(mask, tf.int64)
        fmask = tf.cast(mask, dtype)
        # Update values using mask.
        #feat_labels = imask * label + (1 - imask) * feat_labels
        feat_scores = tf.where(mask, jaccard, feat_scores)

        feat_ymin = fmask * bbox[0] + (1 - fmask) * feat_ymin
        feat_xmin = fmask * bbox[1] + (1 - fmask) * feat_xmin
        feat_ymax = fmask * bbox[2] + (1 - fmask) * feat_ymax
        feat_xmax = fmask * bbox[3] + (1 - fmask) * feat_xmax

        # Check no annotation label: ignore these anchors...
        #interscts = intersection_with_anchors(bbox)
        #mask = tf.logical_and(interscts > ignore_threshold,
        #                     label == no_annotation_label)
        # Replace scores by -1.
        #feat_scores = tf.where(mask, -tf.cast(mask, dtype), feat_scores)

        return [i+1, feat_scores,
                feat_ymin, feat_xmin, feat_ymax, feat_xmax]
    # Main loop definition.
    '''
    i = 0
    [i,feat_scores,
     feat_ymin, feat_xmin,
     feat_ymax, feat_xmax] = tf.while_loop(condition, body,
                                           [i, feat_scores,
                                            feat_ymin, feat_xmin,
                                            feat_ymax, feat_xmax])
    '''
    for i, bbox in enumerate(tf.unstack(bboxes, axis=0)):
        [i,feat_scores,feat_ymin, 
        feat_xmin, feat_ymax, feat_xmax] = body(i, feat_scores,
                                                feat_ymin, feat_xmin, 
                                                feat_ymax, feat_xmax,bbox)
    # Transform to center / size.
    feat_cy = (feat_ymax + feat_ymin) / 2.
    feat_cx = (feat_xmax + feat_xmin) / 2.
    feat_h = feat_ymax - feat_ymin
    feat_w = feat_xmax - feat_xmin
    # Encode features.
    feat_cy = (feat_cy - yref) / href / prior_scaling[0]
    feat_cx = (feat_cx - xref) / wref / prior_scaling[1]
    feat_h = tf.log(feat_h / href) / prior_scaling[2]
    feat_w = tf.log(feat_w / wref) / prior_scaling[3]
    # Use SSD ordering: x / y / w / h instead of ours.
    feat_localizations = tf.stack([feat_cx, feat_cy, feat_w, feat_h], axis=-1)
    return feat_localizations, feat_scores


In [13]:
def tf_text_bboxes_encode(bboxes,
                         anchors,
                         matching_threshold=0.5,
                         prior_scaling=[0.1, 0.1, 0.2, 0.2],
                         dtype=tf.float32,
                         scope='ssd_bboxes_encode'):
    """Encode groundtruth labels and bounding boxes using SSD net anchors.
    Encoding boxes for all feature layers.

    Arguments:
      bboxes: Nx4 Tensor(float) with bboxes relative coordinates;
      anchors: List of Numpy array with layer anchors;
      matching_threshold: Threshold for positive match with groundtruth bboxes;
      prior_scaling: Scaling of encoded coordinates.

    Return:
      (target_labels, target_localizations, target_scores):
        Each element is a list of target Tensors.
    """
    with tf.name_scope(scope):
        target_labels = []
        target_localizations = []
        target_scores = []
        for i, anchors_layer in enumerate(anchors):
            with tf.name_scope('bboxes_encode_block_%i' % i):
                t_loc, t_scores = \
                    tf_text_bboxes_encode_layer(bboxes, anchors_layer,
                                                matching_threshold,
                                               prior_scaling, dtype)
                target_localizations.append(t_loc)
                target_scores.append(t_scores)
        return target_localizations, target_scores

In [14]:
data_dir = '/Users/xiaodiu/Documents/github/projecttextbox/TextBoxes-TensorFlow/data/sythtext/'
file_name = data_dir + '1.tfrecord'
## test if file_name exists  

example = tf.python_io.tf_record_iterator(file_name).next()
image_buffer, label, bboxes, name= parse_example(example)
image,label,bboxes = image_processing(image_buffer, bboxes,label,
                                     train= True, thread_id = 0)
flocalization, fscores = tf_text_bboxes_encode(bboxes,layers_anchors,matching_threshold=0.1)
#print flocalization.shape
#print fscores.shape


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    Image, label, bboxes = sess.run([image, label, bboxes])
    flocalization, fscores = sess.run([flocalization,fscores])
    print label.shape
    print bboxes
    #print name
    #print width
    #print height
    print Image.shape
    print flocalization[0].shape
    for i in range(6):
        print np.where(fscores[i] > 0)
    """
    visualize_bbox(Image, bboxes)
    skio.imshow(Image)
    skio.show()
    """

name SparseTensor(indices=Tensor("ParseSingleExample_1/Slice_Indices_image/name:0", shape=(?, 1), dtype=int64), values=Tensor("ParseSingleExample_1/ParseExample/ParseExample:6", shape=(?,), dtype=string), dense_shape=Tensor("ParseSingleExample_1/Squeeze_Shape_image/name:0", shape=(1,), dtype=int64))
image after decode Tensor("decode_jpeg_1/convert_image:0", shape=(?, ?, 3), dtype=float32)
labels: Tensor("ExpandDims_11:0", shape=(1, ?), dtype=int64) 
(38, 38, 2, 1)
(6,)
(?, 4)


ValueError: Cannot infer num from shape (?, 4)